In [24]:
import py_dss_interface
from typing import List
import math
import json


#you don't have to create busbars, they are already defined in other element
#def create_other_elements(in_data,x):
#    print("cos")   

in_data = {
      '0': {'typ': 'PowerFlowOpenDss Parameters', 'frequency': '50', 'algorithm': 'nr'}, 
      '1': {'typ': 'External Grid0', 'name': 'yRxyd1Zk0AVptIg___QcNK___17', 'firstnumberinid': 0, 'bus': 'j5X72v2I2KgoAc5RDL2y___7', 'vm_pu': '1', 'va_degree': '0', 's_sc_max_mva': '1000000', 's_sc_min_mva': '0', 'rx_max': '0', 'rx_min': '0', 'r0x0_max': '0', 'x0x_max': '0'}, 
      '2': {'typ': 'Bus0', 'name': 'j5X72v2I2KgoAc5RDL2y___7', 'firstnumberinid': 0, 'vn_kv': '115'}, 
      '3': {'typ': 'Bus1', 'name': 'j5X72v2I2KgoAc5RDL2y___11', 'firstnumberinid': 0, 'vn_kv': '115'}, 
      '4': {'typ': 'Load0', 'name': 'j5X72v2I2KgoAc5RDL2y___14', 'firstnumberinid': 0, 'bus': 'j5X72v2I2KgoAc5RDL2y___11', 'p_mw': '60', 'q_mvar': '30', 'const_z_percent': '0', 'const_i_percent': '0', 'sn_mva': '0', 'scaling': '1', 'type': 'Wye'}, 
      '5': {'typ': 'Load1', 'name': 'UYkhRgLtxeo1WOCvvwka___1', 'firstnumberinid': 0, 'bus': 'j5X72v2I2KgoAc5RDL2y___11', 'p_mw': '0', 'q_mvar': '0', 'const_z_percent': '0', 'const_i_percent': '0', 'sn_mva': '0', 'scaling': '1', 'type': 'Wye'}, 
      '6': {'typ': 'Line0', 'name': 'j5X72v2I2KgoAc5RDL2y___6', 'firstnumberinid': 0, 'busFrom': 'j5X72v2I2KgoAc5RDL2y___7', 'busTo': 'j5X72v2I2KgoAc5RDL2y___11', 'length_km': '10', 'parallel': '1', 'df': '1', 'r_ohm_per_km': '0.06', 'x_ohm_per_km': '0.144', 'c_nf_per_km': '144', 'g_us_per_km': '0', 'max_i_ka': '0.588', 'type': 'cs', 'r0_ohm_per_km': '0.1', 'x0_ohm_per_km': '0', 'c0_nf_per_km': '0', 'endtemp_degree': '250'}
      }


frequency = 50
dss = py_dss_interface.DSS()

f=frequency         

In [25]:
#def powerflow(in_data, frequency):

com_frequency = [
        'clear',
        f'set DefaultBaseFrequency={f}',
]
    #dss.text(com_frequency)

BusbarsDict = {} 
nBusbar = int(0)

command_line = [] 
command_load = []
command_externalgrid = []

#tworzę dictionary Busbar i przypisuję nazwom szyn cyfry int (ponieważ tylko one mogą być używane przez py_dss_interface)
#przykład przypisania: {'j5X72v2I2KgoAc5RDL2y___7': 0, 'j5X72v2I2KgoAc5RDL2y___11': 1}
for x in in_data:         
    if "Bus" in in_data[x]['typ']:
        BusbarsDict[in_data[x]['name']]=nBusbar
        print(nBusbar)
        nBusbar=nBusbar+1

print("BusbarsDict: ")                  
print(BusbarsDict) 


for x in in_data:
    if (in_data[x]['typ'].startswith("External Grid")):
        
        name=in_data[x]['name']
        #bus=in_data[x]['bus']

        for key, value in BusbarsDict.items():
            if key == in_data[x]['bus']:
                bus=value

        vm_pu=in_data[x]['vm_pu']
        s_sc_max_mva=eval(in_data[x]['s_sc_max_mva'])
            
        #BasekV musisz wziąć szyny do której jest przyłączana external grid - DO ZROBIENIA
        com_externalgrid = [
            f'New Circuit.{name} bus1={bus} Basekv=110  pu={vm_pu}  mvasc3={s_sc_max_mva}  1000000', 
        ]
        #com_externalgrid.extend(com_externalgrid)
        #dss.text(com_externalgrid)
        
    if (in_data[x]['typ'].startswith("Line")):  

        name=in_data[x]['name']
        #print('Linia name')
        #print(name)

        for key, value in BusbarsDict.items():
            if key == in_data[x]['busFrom']:
                from_bus=value
            if key == in_data[x]['busTo']:
                to_bus=value

        r_ohm_per_km=in_data[x]['r_ohm_per_km']
        x_ohm_per_km=in_data[x]['x_ohm_per_km']
        c_nf_per_km= in_data[x]['c_nf_per_km']
        length_km=in_data[x]['length_km']
        #g_us_per_km= in_data[x]['g_us_per_km']
        r0_ohm_per_km=0 
        x0_ohm_per_km=0
        c0_nf_per_km=0


        #Lines in ohms @ 115 kV
        com_line = [
            f'New Line.{name} phases=3 Bus1={from_bus}  Bus2={to_bus}  R1={r_ohm_per_km}   X1={x_ohm_per_km}    R0={r0_ohm_per_km}   X0={x0_ohm_per_km}   C1={c_nf_per_km}  C0={c0_nf_per_km} Length={length_km} units=km',
        ]
        command_line.extend(com_line)
        #dss.text(com_line)

    if (in_data[x]['typ'].startswith("Load")):

            name=in_data[x]['name']

            for key, value in BusbarsDict.items():
                if key == in_data[x]['bus']:
                    bus=value            
                                   
            p_kw=float(in_data[x]['p_mw'])*1000
            q_kvar=float(in_data[x]['q_mvar'])*1000

            #napięcie weź z przyłączonej szyny 

            com_load = [
                f'New load.{name}  phases=3  bus1={bus}  kV=115  kW={p_kw}  kvar= {q_kvar}  vmin=0.9',
            ]
            command_load.extend(com_load)
            #dss.text(com_load)
    
com_solve = [
        'Set voltagebases=[115]',
        'calcv',
        'solve',        
]

    
com_frequency.extend(com_externalgrid)
com_frequency.extend(command_line)
com_frequency.extend(command_load)
com_frequency.extend(com_solve)

for cmd in com_frequency:
    print(cmd)
    dss.text(cmd) 


class BusbarOut(object):
    def __init__(self, name: str):#, , vm_pu: float, va_degree: float, p_mw: float, q_mvar: float, pf: float, q_p: float):          firstnumberinid: str
            self.name = name
            #self.firstnumberinid = firstnumberinid
            #self.vm_pu = vm_pu
            #self.va_degree = va_degree   
            #self.p_mw = p_mw
            #self.q_mvar = q_mvar  
            #self.pf = pf #p_mw/math.sqrt(math.pow(p_mw,2)+math.pow(q_mvar,2))  
            #self.q_p = q_p
                        
class BusbarsOut(object):
    def __init__(self, busbars: List[BusbarOut]):
        self.busbars = busbars                
                
busbarList = list() 
    





0
1
BusbarsDict: 
{'j5X72v2I2KgoAc5RDL2y___7': 0, 'j5X72v2I2KgoAc5RDL2y___11': 1}
clear
set DefaultBaseFrequency=50
New Circuit.yRxyd1Zk0AVptIg___QcNK___17 bus1=0 Basekv=110  pu=1  mvasc3=1000000  1000000
New Line.j5X72v2I2KgoAc5RDL2y___6 phases=3 Bus1=0  Bus2=1  R1=0.06   X1=0.144    R0=0   X0=0   C1=144  C0=0 Length=10 units=km
New load.j5X72v2I2KgoAc5RDL2y___14  phases=3  bus1=1  kV=115  kW=60000.0  kvar= 30000.0  vmin=0.9
New load.UYkhRgLtxeo1WOCvvwka___1  phases=3  bus1=1  kV=115  kW=0.0  kvar= 0.0  vmin=0.9
Set voltagebases=[115]
calcv
solve


In [26]:
       
for bus in dss.circuit.buses_names:
    dss.circuit.set_active_bus(bus)
    print("tutaj wypisuje")
    print(dss.bus.name)
    #print(bus)

    for key, value in BusbarsDict.items():                
        print('BusbsarsDic')
        print('key: ')
        print(key)
        print('value: ')
        print(value)
        print('dss.bus.name: ')
        print(dss.bus.name)
        print(value == int(dss.bus.name))
        if value == int(dss.bus.name):
            busName=key   
            busbar = BusbarOut(name=busName)#, firstnumberinid = net.bus._get_value(index, 'firstnumberinid'), vm_pu=row['vm_pu'], va_degree=row['va_degree'], p_mw=row['p_mw'], q_mvar=row['q_mvar'], pf = pf, q_p=q_p)      
            busbarList.append(busbar) 
            busbars = BusbarsOut(busbars = busbarList)
            result = {**busbars.__dict__}       
            response = json.dumps(result, default=lambda o: o.__dict__, indent=4)   
    print(response)                                   
    #    return response                 


           
        
        
        
        #U[pu],angle[degree]
        #print(dss.bus.vmag_angle_pu)    
        #dss.circuit.set_active_element(dss.bus.name)
        #print(dss.cktelement.powers)
        #print(dss.circuit.total_power)
    #P[MW]
    #Q[MVar]
    #PF        

tutaj wypisuje
0
BusbsarsDic
key: 
j5X72v2I2KgoAc5RDL2y___7
value: 
0
dss.bus.name: 
0
True
BusbsarsDic
key: 
j5X72v2I2KgoAc5RDL2y___11
value: 
1
dss.bus.name: 
0
False
{
    "busbars": [
        {
            "name": "j5X72v2I2KgoAc5RDL2y___7"
        }
    ]
}
tutaj wypisuje
1
BusbsarsDic
key: 
j5X72v2I2KgoAc5RDL2y___7
value: 
0
dss.bus.name: 
1
False
BusbsarsDic
key: 
j5X72v2I2KgoAc5RDL2y___11
value: 
1
dss.bus.name: 
1
True
{
    "busbars": [
        {
            "name": "j5X72v2I2KgoAc5RDL2y___7"
        },
        {
            "name": "j5X72v2I2KgoAc5RDL2y___11"
        }
    ]
}
